# Sistemas Inteligentes 2021/2022

## Mini-projeto 1: Pacman comilão

<img src="pacman.png" alt="Drawing" style="width: 100px;"/>

## Grupo: 42

### Elementos do Grupo

Número:  53687       Nome:   Ariana Dias   
Número:  53746       Nome:   Andrei Tataru  
Número:  51127       Nome:   Luís Ferreirinha  

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação dos estados

(descreva aqui, textualmente, como decidiu representar os estados em Python; ilustre nas células de código abaixo a representação em Python de um estado à sua escolha)

Para representar-mos os estados em python, decidimos criar uma classe EspacoPacman, onde cada objeto desta classe representa um estado do nosso problema, para este problema em concreto decidimos que estes objetos devem guardar a posição do pacman, o número de pontos acumulados, o tempo desde a ultima captura de uma pastilha, casas visitadas e número de vezes que foram visitadas e finalmente as coordenadas e tipo das pastilhas que ainda se encontram no campo.

- Representação dos Estados:  
    - Posição Pacman: tuple $\left(x,y\right)$ onde $x,y$ são inteiros
    - Pontos: inteiro p
    - Tempo: inteiro t
    - Casas Visitadas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : n\}$, $x,y,n$ são inteiros e $n$ representa o número de visitas a essa casa.
    - Conjunto das Pastilhas: dicionário onde os elementos têm a seguinte forma $\{(x,y) : tipo\}$, $x,y$ são inteiros e tipo é um character que representa o tipo da pastilha


In [205]:
# se definiu uma classe para representar os estados, inclua aqui o código Python correspondente
import copy

class EstadoPacman():
    """Objetos desta classe representam um estado do espaço do problema Pacman comilao
    """

    def __init__(self, posicao_pacman, pastilhas, pontos=0, tempo=0, visitas={}):
        self.pos = posicao_pacman
        self.pontos = pontos
        self.tempo = tempo
        self.pastilhas = copy.deepcopy(pastilhas)
        if visitas == {}:
            self.visitas = {posicao_pacman : 1}
        else:
            self.visitas = copy.deepcopy(visitas)

    def __lt__(self, other):
        return self.pontos < other.pontos
            
    def __eq__(self,other):
        if not isinstance(other, EstadoPacman): return False
        return other.pos == self.pos and other.pontos == self.pontos and other.tempo == self.tempo \
            and other.pastilhas == self.pastilhas and other.visitas == self.visitas
    
    def __hash__(self):
        return hash((self.pos, self.pontos, self.tempo)) + \
               hash(frozenset(self.pastilhas)) + \
               hash(frozenset(self.visitas))

    def __repr__(self):
        return str([self.pos, self.pontos, self.tempo, self.pastilhas, self.visitas])

    def __str__(self):
        return f"Posição Pacman: {self.pos}\nPontos: {self.pontos}\nTempo desde última pastilha: {self.tempo}\
                \nPastilhas restantes: {self.pastilhas}\nCasas Visitadas: {self.visitas}"


Vamos representar o seguinte estado:
-   Pacman está em (1,2)
-   Tem 2 pontos
-   Passaram 2 unidades de tempo desda a última pastilha comida
-   Restam as seguintes pastilhas: {(5,6) : "N", (3,2) : "D"}
-   Foram visitadas as seguintes casas: {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}

In [206]:
# Criamos um objecto do tipo EstadoPacman com os parametros anteriores
pastilhas_exemplo = {(5,6) : "N", (3,2) : "D"}
visitas_exemplo = {(0,0) : 1, (1,0) : 1, (1,1) : 1, (1,2) : 1}
estado_exemplo = EstadoPacman((1,2), pastilhas_exemplo , 2, 2, visitas_exemplo)

Como foi definido o método ```__str__``` para esta classe podemos imprimir o objecto para visualizar melhor o estado

In [207]:
print(estado_exemplo)

Posição Pacman: (1, 2)
Pontos: 2
Tempo desde última pastilha: 2                
Pastilhas restantes: {(5, 6): 'N', (3, 2): 'D'}
Casas Visitadas: {(0, 0): 1, (1, 0): 1, (1, 1): 1, (1, 2): 1}


## Formulação do problema

(explique textualmente como decidiu formular em Python o seu problema)

- Estado inicial:
    
    | EstadoPacman |
    | :---: |
    | Posição: $(x,y)$ |
    | Pastilhas: dict(pastilhas iniciais) |
    | Pontos: 0 |
    | Tempo: 0 |
    | Visitas: {(x,y) : 1} |

<br />

- Estado final:

    | EstadoPacman |
    | :---: |
    | Posição: $(x_2,y_2)$ |
    | Pastilhas: dict(pastilhas restantes) |
    | Pontos: Pontos Objectivo |
    | Tempo: 0 |
    | Visitas: dict(casas visitadas n vezes) |  

<br /> 

- Goal Test: Pontos $\geq$ Pontos Objetivo

- Operadores:  
    - Mover-se numa das quatro direcções ortogonais para as quatro células vizinhas, se não for impedido pelos limites do mundo nem por obstáculos:  
        - Norte: $(x,y) \to (x+1,y)$
        - Sul: $(x,y) \to (x-1,y$
        - Este: $(x,y) \to (x,y+1)$
        - Oeste: $(x,y) \to (x,y-1)$
    - Se a casa onde o pacman se encontra contem uma pastilha, ele come automaticamente a pastilha, removendo esta e adicionando os respetivos pontos ao seu total de pontos:
        - Pontos Totais = Pontos Totais + Pontos da Pastilha

In [208]:
from searchPlus import *
import copy

class PacmanPastilhas(Problem):

    moves = {"N" : (0,-1), "S" : (0,1), "O" : (-1,0), "E" : (1,0)}

    def __init__(self, pos : tuple[int,int], objetivo : int, pastilhas : dict, obstaculos : set, dimensao : int):
        self.initial = EstadoPacman(pos, pastilhas)
        self.goal = objetivo
        self.obstacles = obstaculos
        self.dim = dimensao
        

    def calculate_points(self, time : int, pastilha : str) -> int:
        """Calcula os pontos a serem atribuidos por apanhar uma pastilha,
        utilizando o tempo decorrido desde a ultima pastilha capturada e 
        o tipo da pastilha

        Args:
            time (int): Tempo decorrido desde a ultima captura de uma pastilha
            pastilha (str): Tipo de pastilha

        Returns:
            int: Pontos a serem atribuidos
        """
        if pastilha == "N":
            return 1
        elif pastilha == "D":
            return max(0,5-time)
        else:
            return time
    
    def calculate_position(self, pos : tuple[int,int], direction : str) -> tuple[int,int]:
        """Dado a posicao atual do pacman e uma direcao ortogonal, devolve a posicao
        do pacman apos se mover nessa direcao"""
        return (pos[0]+self.moves[direction][0], pos[1]+self.moves[direction][1])

    def actions(self, state : EstadoPacman) -> list[str]:
        if self.goal_test(state): return []
        return [d for d in self.moves.keys() if self.calculate_position(state.pos, d) not in self.obstacles]

    def result(self, state : EstadoPacman, action : str) -> EstadoPacman:
        new_position = self.calculate_position(state.pos, action)
        new_visitas = copy.deepcopy(state.visitas)
        if new_position in new_visitas.keys():
            new_visitas[new_position] += 1
        else:
            new_visitas[new_position] = 1

        if new_position in state.pastilhas.keys():
            new_points = state.pontos + self.calculate_points(state.tempo+1, state.pastilhas[new_position])
            new_pastilhas = copy.deepcopy(state.pastilhas)
            new_pastilhas.pop(new_position)
            new_time = 0
            return EstadoPacman(new_position, new_pastilhas, new_points, new_time, new_visitas)
        
        return EstadoPacman(new_position, state.pastilhas, state.pontos, state.tempo+1, new_visitas)


    def path_cost(self, cost : int, state, action, next_state : EstadoPacman) -> int:
        return cost+next_state.visitas[next_state.pos]

    def display(self, state : EstadoPacman):
        board = ""
        for y in range(self.dim):
            for x in range(self.dim):
                if state.pos == (x,y):
                    board += "@ "
                elif (x,y) in self.obstacles:
                    board += "# "
                elif (x,y) in state.pastilhas.keys():
                    board += f"{state.pastilhas[(x,y)]} "
                else:
                    board += ". "
            board += "\n"
        print(board)

    def goal_test(self, state : EstadoPacman) -> bool:
        return state.pontos >= self.goal

## Criação de estados e do problema

(Mostrem que o código está a funcionar, construindo instâncias da classe **PacmanPastilhas**, fazendo display dos estados, verificando o teste do estado final, gerando as ações para alguns estados, executando ações a partir de alguns estados e gerando novos estados e mostrando a evolução dos custos; verificando que os estados não se modificam com as ações (são gerados novos estados) e que a igualdade e a comparação entre estados funciona. Mostrem que a execução de sequências de ações está a funcionar bem.)

In [209]:
# Funcoes auxiliares

def line(x, y, dx, dy, length):
    """Uma linha de células de comprimento 'length' começando em (x, y) na direcção (dx, dy)."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

def quadro(x, y, length):
    """Uma moldura quadrada de células de comprimento 'length' começando no topo esquerdo (x, y)."""
    return line(x,y,0,1,length) | line(x+length-1,y,0,1,length) | line(x,y,1,0,length) | line(x,y+length-1,1,0,length)

def exec(p,estado,accoes):
    custo = 0
    for a in accoes:
        seg = p.result(estado,a)
        custo = p.path_cost(custo,estado,a,seg)
        estado = seg
    p.display(estado)
    print('Custo:',custo)
    print('Goal?',p.goal_test(estado))
    return (estado,custo)

Para demonstrar o funcionamento do nosso código vamos definir duas instâncias de PacmanPastilhas para demonstrar as funcionalidades da classe.
   
- Primeiro vamos definir um conjunto de obstaculos e das pastilhas para o primeiro mundo

In [210]:
# Mundo 1 - dimensao 10
fronteira1 = quadro(0,0,10)
obs1_1 = line(2,2,1,0,5)
obs1_2 = line(7,7,0,-1,6)
mundo_1 = fronteira1 | obs1_1 | obs1_2

pastilhas_1 = {(5,5) : "C", (4,7) : "N", (1,3) : "D"}

Vamos iniciar o estado com o pacman na posição $(1,1)$ e com objetivo 5 pontos

In [211]:
p1 = PacmanPastilhas((1,1),5,pastilhas_1,mundo_1,10) # Mundo 1

Vamos imprir o estado inicial do problema

In [212]:
# Representacao dos parametros do estado
print(p1.initial)
print() # linha vazia para melhorar visibilidade
# Representacao do estado
p1.display(p1.initial)

Posição Pacman: (1, 1)
Pontos: 0
Tempo desde última pastilha: 0                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N', (1, 3): 'D'}
Casas Visitadas: {(1, 1): 1}

# # # # # # # # # # 
# @ . . . . . . . # 
# . # # # # # # . # 
# D . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 



Podemos também verificar que o estado inicial não é solução do problema

In [213]:
print(p1.goal_test(p1.initial))

False


Vamos agora ver as ações disponíveis e aplicar algumas ao nosso problema de forma a gerar uma pequena sequência de estados

- Ações disponíveis a partir do estado inicial:

In [214]:
print(p1.actions(p1.initial))

['S', 'E']


- Vamos aplicar a ação "S" (mover uma casa para baixo), ver o estado resultante utilizando p1.display(estado) e vamos também guardar o custo

In [215]:
p1_s2 = p1.result(p1.initial, "S")
p1.display(p1_s2)
custo = p1.path_cost(0, p1.initial, "S", p1_s2)
print(f"custo: {custo}")
print(f"\nAções:\n {p1.actions(p1_s2)}")

# # # # # # # # # # 
# . . . . . . . . # 
# @ # # # # # # . # 
# D . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

custo: 1

Ações:
 ['N', 'S']


- Vamos agora mover o pacman outra casa para baixo de forma a capturar a pastilha do tipo D

In [216]:
p1_s3 = p1.result(p1_s2, "S")
p1.display(p1_s3)
custo = p1.path_cost(custo, p1_s2, "S", p1_s3)
print(f"custo: {custo}")
print(f"\nAções:\n {p1.actions(p1_s3)}")

# # # # # # # # # # 
# . . . . . . . . # 
# . # # # # # # . # 
# @ . . . . . # . # 
# . . . . . . # . # 
# . . . . C . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

custo: 2

Ações:
 ['N', 'S', 'E']


- Se voltarmos uma casa atrás o custo vai acrescentar 2 porque ja visitamos aquela casa uma vez antes

In [217]:
cost_test1 = p1.result(p1_s3, "N")
custo2 = p1.path_cost(custo, p1_s3, "N", cost_test1)
print(f"custo: {custo2}")

custo: 4


- Podemos ver o número de pontos que o pacman tem ao fazer print do estado com print(estado), como os estados não são modificados após a aplicação de uma ação podemos visualizar e comparar estados anteriores

In [218]:
print("Segundo estado (p1_s2):")
print(p1_s2)
print()
print("Terceiro estado (p1_s3):")
print(p1_s3)

print("\nComparações:")
print("p1_s2 == p1_s3 ?")
print(p1_s2 == p1_s3)
print("p1_s2 < p1_s3 ?")
print(p1_s2 < p1_s3)
print("\n p1_s2 tem menos pontos do que p1_s3 logo é menor")


Segundo estado (p1_s2):
Posição Pacman: (1, 2)
Pontos: 0
Tempo desde última pastilha: 1                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N', (1, 3): 'D'}
Casas Visitadas: {(1, 1): 1, (1, 2): 1}

Terceiro estado (p1_s3):
Posição Pacman: (1, 3)
Pontos: 3
Tempo desde última pastilha: 0                
Pastilhas restantes: {(5, 5): 'C', (4, 7): 'N'}
Casas Visitadas: {(1, 1): 1, (1, 2): 1, (1, 3): 1}

Comparações:
p1_s2 == p1_s3 ?
False
p1_s2 < p1_s3 ?
True

 p1_s2 tem menos pontos do que p1_s3 logo é menor


- Podemos também criar uma instância de EstadoPacman com os mesmos atributos que p1_s2 e mostrar que são iguais

In [219]:
eg1 = EstadoPacman(p1_s2.pos, p1_s2.pastilhas, p1_s2.pontos, p1_s2.tempo, p1_s2.visitas)
print(eg1.pastilhas == p1_s2.pastilhas)

True


- Vamos agora aplicar uma sequência de ações que nos faz chegar ao estado final

In [220]:
# Seq
actions = ["S"]*2 + ["E"]*4 + ["S"]*2
p1_final = exec(p1, p1.initial, actions)
p1.actions(p1_final[0])

# # # # # # # # # # 
# . . . . . . . . # 
# . # # # # # # . # 
# . . . . . . # . # 
# . . . . . . # . # 
# . . . . @ . # . # 
# . . . . . . # . # 
# . . . N . . # . # 
# . . . . . . . . # 
# # # # # # # # # # 

Custo: 8
Goal? True


[]

- Finalmento, vamos criar uma segunda instância de PacmanPastilhas para demonstrar um problema com um mundo de dimensão diferente e com obstáculos diferentes.

In [221]:
fronteira2 = quadro(0,0,15)
obs2_1 = line(2,1,0,1,12)
obs2_2 = line(2,12,1,0,11)
obs2_3 = line(5,3,0,1,9)
obs2_4 = line(10,1,0,1,10)
mundo_2 = fronteira2 | obs2_1 | obs2_2 | obs2_3 | obs2_4

pastilhas_2 = {(3,4) : "C", (12,1) : "D", (1,3) : "D", (1,13) : "N", (12,3) : "N"}

p2 = PacmanPastilhas((1,1),8,pastilhas_2,mundo_2,15) # Mundo 2

# Representacao dos parametros do estado
print(p2.initial)
print()
# Representacao do estado
p2.display(p2.initial)

Posição Pacman: (1, 1)
Pontos: 0
Tempo desde última pastilha: 0                
Pastilhas restantes: {(3, 4): 'C', (12, 1): 'D', (1, 3): 'D', (1, 13): 'N', (12, 3): 'N'}
Casas Visitadas: {(1, 1): 1}

# # # # # # # # # # # # # # # 
# @ # . . . . . . . # . D . # 
# . # . . . . . . . # . . . # 
# D # . . # . . . . # . N . # 
# . # C . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . . . . . # 
# . # # # # # # # # # # # . # 
# N . . . . . . . . . . . . # 
# # # # # # # # # # # # # # # 



In [226]:
solution = ["S"]*12+["E"]*12+["N"]*10+["O"]+["N"]*2
p2_final = exec(p2, p2.initial, solution)

# # # # # # # # # # # # # # # 
# . # . . . . . . . # . @ . # 
# . # . . . . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # C . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . # . . . # 
# . # . . # . . . . . . . . # 
# . # # # # # # # # # # # . # 
# . . . . . . . . . . . . . # 
# # # # # # # # # # # # # # # 

Custo: 37
Goal? True


## Teste de procura de solução

(utilização de algoritmos de procura aprendidos nas aulas e comparação dos resultados ao nível de tempo de execução e solução obtida; comente aqui os resultados obtidos e o que observa)

In [ ]:
# código de aplicação dos algoritmos